In [1]:
from threeML import *
import matplotlib.pyplot as plt
import numpy as np
import pickle

00:39:29 WARNING   The naima package is not available. Models that depend on it will not be         ]8;id=320371;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=336973;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/functions/functions_1D/functions.py#48\48]8;;\
                  available                                                                                        

         WARNING   The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it  ]8;id=918584;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/functions/functions_1D/functions.py\functions.py]8;;\:]8;id=844851;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/functions/functions_1D/functions.py#69\69]8;;\
                  will not be available.                                                                           

/home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


00:39:30 WARNING   The ebltable package is not available. Models that depend on it will not be     ]8;id=916146;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/functions/functions_1D/absorption.py\absorption.py]8;;\:]8;id=501051;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/functions/functions_1D/absorption.py#36\36]8;;\
                  available                                                                                        

         WARNING   We have set the min_value of K to 1e-99 because there was a postive transform   ]8;id=37159;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/core/parameter.py\parameter.py]8;;\:]8;id=59075;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/astromodels/core/parameter.py#704\704]8;;\

...

         WARNING   Could not import plugin FermiLATLike.py. Do you have the relative instrument     ]8;id=898080;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=36204;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py#144\144]8;;\
                  software installed and configured?                                                               

         WARNING   No fermitools installed                                              ]8;id=53450;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py\lat_transient_builder.py]8;;\:]8;id=560580;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/data_builders/fermi/lat_transient_builder.py#44\44]8;;\

         WARNING   Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=417536;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=160768;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for optimal         ]8;id=107939;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=925517;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

         WARNING   Env. variable NUMEXPR_NUM_THREADS is not set. Please set it to 1 for optimal     ]8;id=319029;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py\__init__.py]8;;\:]8;id=86510;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/__init__.py#387\387]8;;\
                  performances in 3ML                                                                              

In [2]:
def powerlaw(piv=100):
    spec = Powerlaw()
    ps = PointSource('s',l=0,b=0,spectral_shape=spec)
    ps_model = Model(ps)
    ps_model.s.spectrum.main.Powerlaw.piv = piv
    
    ps_model.s.spectrum.main.Powerlaw.K.prior = Log_uniform_prior(lower_bound=1e-12, upper_bound=1e0)
    ps_model.s.spectrum.main.Powerlaw.index.prior = Uniform_prior(lower_bound=-5, upper_bound=0)
    
    return ps_model

def broken_powerlaw(piv=100):
    spec = SmoothlyBrokenPowerLaw()
    ps = PointSource('s',l=0,b=0,spectral_shape=spec)
    ps_model = Model(ps)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.pivot = piv
    
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.alpha.min_value = -3
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.K.max_value = 1
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_energy.max_value=1000
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_energy.min_value=1
    
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_scale.free = True
    
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.K.prior = Log_uniform_prior(lower_bound=5e-4, upper_bound=1e-3)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.alpha.prior = Uniform_prior(lower_bound=-2.2, upper_bound=-1.8)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.beta.prior = Uniform_prior(lower_bound=-5.0, upper_bound=-1.9)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_energy.prior = Log_uniform_prior(lower_bound=10, upper_bound=500)
    ps_model.s.spectrum.main.SmoothlyBrokenPowerLaw.break_scale.prior = Uniform_prior(lower_bound=0.0, upper_bound=1.5)
    
    
    ps_model.display(complete=True)
    return ps_model
    

In [21]:
def data_crab(e_range="30-400"):
    d = OGIPLike("crab",
                observation='spectra_Crab_Nebula.fits',
                response='spectral_response.rmf.fits')
    
    d.set_active_measurements(e_range)
    ps_data = DataList(d)
    return ps_data
    
def data_pulsar(e_range="30-400"):
    d = OGIPLike("crab",
                observation='spectra_A0535+26a.fits',
                response='spectral_response.rmf.fits')
    
    d.set_active_measurements(e_range)
    ps_data = DataList(d)
    return ps_data

In [4]:
def fit_data_model(data, model):
    ps_jl = JointLikelihood(model, data)
    best_fit_parameters_ps, likelihood_values_ps = ps_jl.fit()
    ps_jl.restore_best_fit()
    
    val = np.array(best_fit_parameters_ps["value"])
    err = np.array(best_fit_parameters_ps["error"])
    cor = ps_jl.correlation_matrix
    cov = cor * err[:, np.newaxis] * err[np.newaxis, :]
    
    return val, cov, ps_jl

def bayes_analysis(data, model):
    bayes_analysis = BayesianAnalysis(model, data)
    bayes_analysis.set_sampler("multinest")
    bayes_analysis.sampler.setup(n_live_points=800, resume=False, auto_clean=True)
    bayes_analysis.sample()
    
    return bayes_analysis.results._values, bayes_analysis.results.estimate_covariance_matrix()

In [5]:
def save_results(val, cov, name):
    with open(f"{name}.pickle", "wb") as f:
        pickle.dump((val, cov),f)

In [6]:
def low_energy_pl():
    piv = 100
    e_range="30-81.5"
    model = powerlaw(piv)
    data = data_crab(e_range)
    val, cov = bayes_analysis(data, model)
    save_results(val, cov, "crab_low_energy_pl_fit")
    
def brk_pl():
    piv = 100
    e_range="30-400"
    model = broken_powerlaw(piv)
    data = data_crab(e_range)
    val, cov = bayes_analysis(data, model)
    save_results(val, cov, "crab_brk_pl_fit")
    
def pulsar_pl():
    piv = 100
    e_range="30-400"
    model = powerlaw(piv)
    data = data_pulsar(e_range)
    val, cov = bayes_analysis(data, model)
    save_results(val, cov, "pulsar_pl_fit")

In [68]:
piv = 100
e_range="30-81.5"
model = powerlaw(piv)
data = data_crab(e_range)

bayes_analysis = BayesianAnalysis(model, data)
bayes_analysis.set_sampler("multinest")
bayes_analysis.sampler.setup(n_live_points=800, resume=False, auto_clean=True)
bayes_analysis.sample()
val, cov = bayes_analysis.results._values, bayes_analysis.results.estimate_covariance_matrix()
# display_spectrum_model_counts(bayes_analysis)

01:18:34 WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=459733;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=695489;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=943108;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=844023;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=859479;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=708015;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=252946;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=926840;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: gaussian                                                      ]8;id=885512;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=919298;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#470\470]8;;\

...

,-log(posterior)
crab,-1576.953401
total,-1576.953401


Values of statistical measures:

,statistical measures
AIC,3158.222591
BIC,3161.333946
DIC,3157.818203
PDIC,1.955597
log(Z),-695.000687


         INFO      deleting the chain directory chains                                     ]8;id=594263;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py\multinest_sampler.py]8;;\:]8;id=67362;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py#255\255]8;;\

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mFailed in nopython mode pipeline (step: native lowering)
Method '__array__' is not supported.[0m
[0m[1m- Resolution failure for non-literal arguments:
[1mNone[0m
[0m[0m
[0m[1mDuring: resolving callee type: BoundFunction((<class 'numba.core.types.misc.ClassInstanceType'>, 'append') for instance.jitclass._Vector#7f421d176ee0<n:uint64,m:uint64,full_arr:array(float64, 1d, A)>)[0m
[0m[1mDuring: typing of call at /home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/binner.py (659)
[0m
[1m
File "../../../../.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/binner.py", line 659:[0m
[1mdef _rebin_vector_float(vector, start, stop, mask, N):
    <source elided>

[1m        rebinned_vector.append(np.sum(vector[start[n] : stop[n]]))
[0m        [1m^[0m[0m


In [64]:
low_energy_pl()

         WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=442405;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=561834;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=623362;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=110143;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=33378;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=173480;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=963426;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=546614;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: gaussian                                                      ]8;id=105508;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=365161;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#470\470]8;;\

         INFO      - background: None                                                           ]8;id=810558;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=285296;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#471\471]8;;\

         INFO      Range 30-81.5 translates to channels 4-16                                   ]8;id=552127;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=803833;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#1210\1210]8;;\

TypeError: 'BayesianAnalysis' object is not callable

In [ ]:
brk_pl()

Model summary:
==============

                  N
Point sources     1
Extended sources  0
Particle sources  0

Free parameters (5):
--------------------

                                          value min_value max_value  \
s.spectrum.main.SmoothlyBrokenPowerLaw.K    1.0       0.0       1.0   
s...alpha                                  -1.0      -3.0       2.0   
s...break_energy                          300.0       1.0    1000.0   
s...break_scale                             0.5       0.0      10.0   
s...beta                                   -2.0      -5.0      -1.6   

                                                    unit  
s.spectrum.main.SmoothlyBrokenPowerLaw.K  keV-1 s-1 cm-2  
s...alpha                                                 
s...break_energy                                     keV  
s...break_scale                                           
s...beta                                                  

Fixed parameters (3):
---------------------

              value min_value max_value unit
s.position.l    0.0       0.0     360.0  deg
s.position.b    0.0     -90.0      90.0  deg
s...pivot     100.0      None      None  keV

Properties (0):
--------------------

(none)


Linked parameters (0):
----------------------

(none)

Independent variables:
----------------------

(none)

Linked functions (0):
----------------------

(none)

         WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=728493;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=293390;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=718666;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=227355;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=763930;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=759420;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=557630;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=707057;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

...

Values of -log(posterior) at the minimum:

,-log(posterior)
crab,-21.291879
total,-21.291879


Values of statistical measures:

,statistical measures
AIC,54.805981
BIC,60.066296
DIC,36.401451
PDIC,-11.054639
log(Z),-17.977181


         INFO      deleting the chain directory chains                                     ]8;id=722130;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py\multinest_sampler.py]8;;\:]8;id=619804;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py#255\255]8;;\

In [ ]:
pulsar_pl()

         WARNING   Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.                  ]8;id=8189;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=72848;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#274\274]8;;\

         WARNING   FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.    ]8;id=215932;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py\pha_spectrum.py]8;;\:]8;id=24921;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/spectrum/pha_spectrum.py#423\423]8;;\

         WARNING   The response is in an extension called MATRIX, which usually means you also need ]8;id=457816;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py\response.py]8;;\:]8;id=322336;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/utils/OGIP/response.py#459\459]8;;\
                  an ancillary file (ARF) which you didn't provide. You should refer to the                        
                  documentation  of the instrument and make sure you don't need an ARF.                            

         INFO      Auto-probed noise models:                                                    ]8;id=942236;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=217981;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#469\469]8;;\

         INFO      - observation: gaussian                                                      ]8;id=754079;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py\SpectrumLike.py]8;;\:]8;id=188881;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/plugins/SpectrumLike.py#470\470]8;;\

...

Values of -log(posterior) at the minimum:

,-log(posterior)
crab,-377.48967
total,-377.48967


Values of statistical measures:

,statistical measures
AIC,759.379341
BIC,761.972356
DIC,758.213923
PDIC,-6.598263
log(Z),-176.415433


         INFO      deleting the chain directory chains                                     ]8;id=357156;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py\multinest_sampler.py]8;;\:]8;id=670820;file:///home/moej56153/.pyenv/versions/3.9.13/envs/SMF/lib/python3.9/site-packages/threeML/bayesian/multinest_sampler.py#255\255]8;;\

In [23]:
import astropy.io.fits as fits
from astropy.table import Table

In [46]:
with fits.open(f"spectra_Crab_Nebula.fits") as file:
    t = Table.read(file[1])
    t = Table.read(file[2])
    print(file)

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f41d58b4be0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f41d99e37c0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f41d99e3700>]


In [47]:
t

CHANNEL,RATE,STAT_ERR,SYS_ERR,QUALITY,GROUPING,DFLUX,DFLUX_ERR
,ct / (s cm2),ct / (s cm2),,,,counts/(s.cm**2.keV),counts/(s.cm**2.keV)
int16,float32,float32,float32,int16,int16,float32,float32
1,0.033349097,0.00019796228,0.0,0,1,0.022232732,0.00013197486
2,0.036774393,0.00020662209,0.0,0,1,0.018387197,0.000103311046
3,0.028117191,0.00018929022,0.0,0,1,0.0140585955,9.464511e-05
4,0.023199566,0.00010351832,0.0,0,1,0.011599783,5.175916e-05
5,0.024707027,8.045052e-05,0.0,0,1,0.0098828105,3.2180207e-05
6,0.02096007,6.871679e-05,0.0,0,1,0.008384028,2.7486718e-05
7,0.021404753,6.601562e-05,0.0,0,1,0.0071349177,2.2005208e-05
8,0.01821142,5.8979163e-05,0.0,0,1,0.006070473,1.9659721e-05


In [11]:
t[5:17]

CHANNEL,RATE,STAT_ERR,SYS_ERR,QUALITY,GROUPING,DFLUX,DFLUX_ERR
,ct / (s cm2),ct / (s cm2),,,,counts/(s.cm**2.keV),counts/(s.cm**2.keV)
int16,float32,float32,float32,int16,int16,float32,float32
6,0.02096007,6.871679e-05,0.0,0,1,0.008384028,2.7486718e-05
7,0.021404753,6.601562e-05,0.0,0,1,0.0071349177,2.2005208e-05
8,0.01821142,5.8979163e-05,0.0,0,1,0.006070473,1.9659721e-05
9,0.017911926,5.7738045e-05,0.0,0,1,0.005117693,1.6496584e-05
10,0.015318312,5.3898068e-05,0.0,0,1,0.0043766606,1.5399448e-05
11,0.01476177,5.621496e-05,0.0,0,1,0.0036904425,1.405374e-05
12,0.013591006,0.00010458787,0.0,0,1,0.0030202237,2.3241748e-05
13,0.011554832,0.00011428751,0.0,0,1,0.0025677404,2.5397225e-05


In [12]:
smf_bins = np.array([20.0, 21.5, 23.5, 25.5, 27.5, 30.0, 32.5, 35.5, 38.5, 42.0, 45.5, 49.5, 54.0, 58.5, 63.5,
                         69.0, 75.0, 81.5, 89.0, 96.5, 105.0, 114.0, 124.0, 134.5, 146.0, 159.0, 172.5, 187.5, 204.0,
                         221.5, 240.5, 261.5, 284.0, 308.5, 335.5, 364.5, 396.0, 430.0, 467.5, 508.0, 514.0, 600.0,])

In [14]:
len(smf_bins)

42

In [69]:
n_bins = smf_bins#[5:18]
n_bins

array([ 20. ,  21.5,  23.5,  25.5,  27.5,  30. ,  32.5,  35.5,  38.5,
        42. ,  45.5,  49.5,  54. ,  58.5,  63.5,  69. ,  75. ,  81.5,
        89. ,  96.5, 105. , 114. , 124. , 134.5, 146. , 159. , 172.5,
       187.5, 204. , 221.5, 240.5, 261.5, 284. , 308.5, 335.5, 364.5,
       396. , 430. , 467.5, 508. , 514. , 600. ])

In [70]:
val

array([ 6.53505159e-04, -2.17184769e+00])

In [76]:
model.s.spectrum.main.Powerlaw.K = 7.7e-4
model.s.spectrum.main.Powerlaw.index = -2.05
model.s.spectrum.main.Powerlaw.K = val[0]
model.s.spectrum.main.Powerlaw.index = val[1]

In [77]:
model.s(n_bins[1:])

array([1.84115779e-02, 1.51772760e-02, 1.27102286e-02, 1.07878032e-02,
       8.93021710e-03, 7.50523019e-03, 6.19561659e-03, 5.19475630e-03,
       4.30025471e-03, 3.61406683e-03, 3.00967787e-03, 2.49143187e-03,
       2.09387627e-03, 1.75224347e-03, 1.46299975e-03, 1.22066622e-03,
       1.01906330e-03, 8.41717231e-04, 7.06079072e-04, 5.87799335e-04,
       4.91654876e-04, 4.09591890e-04, 3.43307921e-04, 2.87276202e-04,
       2.38696000e-04, 1.99976591e-04, 1.66852162e-04, 1.38924696e-04,
       1.16184951e-04, 9.71684135e-05, 8.10147483e-05, 6.77190146e-05,
       5.65797875e-05, 4.71546933e-05, 3.93846990e-05, 3.28962285e-05,
       2.75075567e-05, 2.29395749e-05, 1.91522828e-05, 1.86700470e-05,
       1.33420641e-05])

In [78]:
def powerlaw_binned_spectrum(energy_bins, spectrum):
    assert np.amin(energy_bins) > 0, "All energy bin values must be greater 0"
    assert np.amin(spectrum) > 0, "All spectrum values must be greater 0"
    
    B = np.log(spectrum[1:] / spectrum[:-1]) / np.log(energy_bins[1:] / energy_bins[:-1])
    A = spectrum[:-1] / (energy_bins[:-1] ** B)
    
    C = B + 1.
    # prevent rounding errors
    C[np.abs(C) < 1e-7] = 0
    
    regular = np.nonzero(C)[0]
    non_regular = []
    for i in range(len(energy_bins) - 1):
        if not i in regular:
            non_regular.append(i)
    # non_regular = np.array(non_regular)
    
    # print(regular)
    
    binned_spectrum = np.zeros(len(energy_bins) - 1)
    binned_spectrum[regular] = A[regular] / (C[regular]) * (energy_bins[regular+1]**(C[regular]) - energy_bins[regular]**(C[regular]))
    # binned_spectrum[non_regular] = A[non_regular] * (np.log(energy_bins[non_regular+1]) - np.log(energy_bins[non_regular]))
    
    return binned_spectrum

In [79]:
(model.s(n_bins[1:])+model.s(n_bins[:-1]))/2 * (n_bins[1:]-n_bins[:-1])

array([0.0299659 , 0.03358885, 0.0278875 , 0.02349803, 0.02464753,
       0.02054431, 0.02055127, 0.01708556, 0.01661627, 0.01385006,
       0.01324749, 0.0123775 , 0.01031694, 0.0096153 , 0.00884192,
       0.008051  , 0.00727912, 0.00697793, 0.00580424, 0.00549898,
       0.00485754, 0.00450623, 0.00395272, 0.00362586, 0.00341882,
       0.00296104, 0.00275122, 0.00252266, 0.00223221, 0.00202686,
       0.00187092, 0.00167325, 0.00152266, 0.00140042, 0.00125482,
       0.00113842, 0.00102686, 0.00094588, 0.00085236, 0.00011347,
       0.00137652])

In [80]:
powerlaw_binned_spectrum(np.array(n_bins), np.array(model.s(n_bins)))

array([0.02987616, 0.03343689, 0.02778105, 0.02342134, 0.0245408 ,
       0.02046898, 0.02045965, 0.01702121, 0.01654432, 0.01379928,
       0.01319368, 0.0123239 , 0.01027912, 0.00957829, 0.00880701,
       0.00801897, 0.00725035, 0.006947  , 0.00578249, 0.00547657,
       0.00483875, 0.00448801, 0.00393778, 0.00361189, 0.00340459,
       0.00294979, 0.00274027, 0.00251239, 0.00222356, 0.002019  ,
       0.00186342, 0.00166673, 0.0015167 , 0.00139478, 0.00124989,
       0.00113395, 0.00102288, 0.0009421 , 0.00084899, 0.00011346,
       0.00135783])

In [52]:
(n_bins[1:]-n_bins[:-1])

array([2.5, 3. , 3. , 3.5, 3.5, 4. , 4.5, 4.5, 5. , 5.5, 6. , 6.5])